<a href="https://colab.research.google.com/github/IgnatiusEzeani/spatio-textual/blob/main/spatio_textual_package_a_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducing `spatio-textual` Python Library

**spatio-textual**: a Python package for spatial entity recognition and verb relation extraction from text created by the [Spatial Narratives Project](https://spacetimenarratives.github.io/) and designed to support spatio-textual annotation, analysis and visualization in digital humanities projects, with initial applications to:

- the *Corpus of Lake District Writing* (CLDW)
- Holocaust survivors' testimonies (e.g., USC Shoah Foundation archives)

This package leverages spaCy and gazetteer-based classification to identify and label spatial entities such as cities, countries, camps, and geographic nouns, and also extracts action-verb contexts involving these entities.


---
## Setting up
Download `en_core_web_trf` spaCy model and install `spatio-textual`



In [ ]:
# !python -m spacy download en_core_web_trf
!pip install -q git+https://github.com/SpaceTimeNarratives/spatio-textual.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import spatio_textual
from spatio_textual.annotate import annotate_text
text ="""EB: It's 50 years [INAUDIBLE]. My parents were both handicapped.
They couldn't speak, and they couldn't hear. So as a small child, I learned
sign language. And I didn't think it was anything special because it came natural to me.
And I have an older sister and an other brother.
My parents-- my father was a tailor. My mother was a housekeeper.
But she did some sewing at home. We were poor, but [PAUSES FOR 3 SECONDS]
my parents-- we were their pride. [PAUSES FOR 7 SECONDS] I [PAUSES FOR 3 SECONDS]
was born in 1927. And in 1932, I started going to public school.
1933, I had to leave the public school, because I was one of the very few Jewish
children there. And I was sent to a Jewish girl school. The school was about
45 minutes to walk. So my mother used to walk with me every morning and pick me
up from school. Wherever my parents had to do something in an office,
or shopping, I went with them to interpret. Because my brother and sister were
already old enough to go to work. [PAUSES FOR 3 SECONDS] And in doing-- after 1933,
of course, we were very conscious that we were Jews. And people-- we had only
Jewish friends, actually. Near our Jewish school was a Gentile-- a regular public
school. And those kids would throw rocks at us, in the winter would cover stones
with snow and throw at us, and yell at us, and call us names.
But [PAUSES FOR 3 SECONDS] didn't really affect me. I guess I was young.
I didn't really understand what was going on. """
result = annotate_text(text)
result

{'entities': [{'start_char': 0, 'token': 'EB', 'tag': 'PERSON'},
  {'start_char': 9, 'token': '50 years', 'tag': 'DATE'},
  {'start_char': 18, 'token': '[INAUDIBLE]', 'tag': 'NON-VERBAL'},
  {'start_char': 34, 'token': 'parents', 'tag': 'FAMILY'},
  {'start_char': 124, 'token': 'child', 'tag': 'FAMILY'},
  {'start_char': 250, 'token': 'sister', 'tag': 'FAMILY'},
  {'start_char': 270, 'token': 'brother', 'tag': 'FAMILY'},
  {'start_char': 295, 'token': 'father', 'tag': 'FAMILY'},
  {'start_char': 319, 'token': 'mother', 'tag': 'FAMILY'},
  {'start_char': 396, 'token': '[PAUSES FOR 3 SECONDS]', 'tag': 'NON-VERBAL'},
  {'start_char': 453, 'token': '[PAUSES FOR 7 SECONDS]', 'tag': 'NON-VERBAL'},
  {'start_char': 478, 'token': '[PAUSES FOR 3 SECONDS]', 'tag': 'NON-VERBAL'},
  {'start_char': 513, 'token': '1927', 'tag': 'DATE'},
  {'start_char': 526, 'token': '1932', 'tag': 'DATE'},
  {'start_char': 558, 'token': 'school', 'tag': 'GEONOUN'},
  {'start_char': 566, 'token': '1933', 'tag': 'DAT

In [ ]:
# text = "Anne Frank was taken from Amsterdam to Auschwitz."
result = annotate_text(text)
result

{'entities': [{'start_char': 0, 'token': 'EB', 'tag': 'PERSON'},
  {'start_char': 9, 'token': '50 years', 'tag': 'DATE'},
  {'start_char': 18, 'token': '[INAUDIBLE]', 'tag': 'NON-VERBAL'},
  {'start_char': 34, 'token': 'parents', 'tag': 'FAMILY'},
  {'start_char': 124, 'token': 'child', 'tag': 'FAMILY'},
  {'start_char': 250, 'token': 'sister', 'tag': 'FAMILY'},
  {'start_char': 270, 'token': 'brother', 'tag': 'FAMILY'},
  {'start_char': 295, 'token': 'father', 'tag': 'FAMILY'},
  {'start_char': 319, 'token': 'mother', 'tag': 'FAMILY'},
  {'start_char': 396, 'token': '[PAUSES FOR 3 SECONDS]', 'tag': 'NON-VERBAL'},
  {'start_char': 453, 'token': '[PAUSES FOR 7 SECONDS]', 'tag': 'NON-VERBAL'},
  {'start_char': 478, 'token': '[PAUSES FOR 3 SECONDS]', 'tag': 'NON-VERBAL'},
  {'start_char': 513, 'token': '1927', 'tag': 'DATE'},
  {'start_char': 526, 'token': '1932', 'tag': 'DATE'},
  {'start_char': 558, 'token': 'school', 'tag': 'GEONOUN'},
  {'start_char': 566, 'token': '1933', 'tag': 'DAT

In [ ]:
pip uninstall spatio_textual

Found existing installation: spatio-textual 0.1.0
Uninstalling spatio-textual-0.1.0:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/spatio_textual-0.1.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/spatio_textual/*
    /usr/local/lib/python3.11/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.11/dist-packages/tests/cloud_database_test.py
    /usr/local/lib/python3.11/dist-packages/tests/conversion_test.py
    /usr/local/lib/python3.11/dist-packages/tests/graph_entities_test.py
    /usr/local/lib/python3.11/dist-packages/tests/graph_server_test.py
    /usr/local/lib/python3.11/dist-packages/tests/magics_test.py
    /usr/local/lib/python3.11/dist-packages/tests/node_expansion_test.py
    /usr/local/lib/python3.11/dist-packages/tests/sample_notebook_test.py
    /usr/local/lib/python3.11/dist-packages/tests/schema_manager_test.py
    /usr/local/lib/python3.11/dist-packages/tests/server_test.py
    /usr/local/lib/python3.